# Computing Loss Function and Gradients

In this notebook, we will finalize the training workflow for the **PQC** by defining the *Loss function* and the corresponding gradient computations. This is the last mandatory step before training the **PQC**.

We already have the following components:

* A dataset (generated using *15_qml4var_DataSets.ipynb*).
* A PQC architecture (from *16_qml4var_BuildPQC.ipynb*).
* A PQC evaluation workflow (from *17_qml4var_pqc_evaluation.ipynb*), which allows us to compute the output of the PQC for given parameters $\theta$ and input **features**.


Before explaining what *Loss function* we will use and how to implement it using the available functions, we need to get some data and configure a **PQC**.

In [ ]:
import sys 
import json
sys.path.append("../../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

Before explaining the loss function we are going to create a dataset and a **PQC**

## 1. Get some dataset

First, some dataset for evaluating the **PQC** is needed. A random dataset will be generated using the *create_random_data* from **benchmark.qml4var.data_sets** module

In [ ]:
from benchmark.qml4var.data_sets import create_random_data

In [ ]:
cfg_random = {
    "n_points_train": 10, 
    "n_points_test" : 100,
    "minval" : -np.pi,
    "maxval" : np.pi,
    "features_number" : 1
}
x_train, y_train, x_test, y_test = create_random_data(
    **cfg_random
)
plt.plot(x_train, y_train, "o")
plt.plot(x_test, y_test, "-")
plt.xlabel("Domain")
plt.ylabel("CDF")
plt.legend(["Training Dataset", "Testing Dataset"])
plt.title("Random Dataset")

## 2. Build the PQC

The *hardware_efficient_ansatz* and the *z_observable* from **QQuantLib.qml4var.architectures** modules will be used to build the **PQC**. Additionally, the *normalize_data* function will be used for data normalization between $\frac{-\pi}{2}$ and $\frac{\pi}{2}$.



In [ ]:
from QQuantLib.qml4var.architectures import hardware_efficient_ansatz, z_observable, normalize_data

In [ ]:
pqc_cfg = {
    "features_number" : cfg_random["features_number"],
    "n_qubits_by_feature" : 2,
    "n_layers": 3    
}
# Normalization function
base_frecuency, shift_feature = normalize_data(
    [cfg_random["minval"]] * cfg_random["features_number"],
    [cfg_random["maxval"]] * cfg_random["features_number"],
    [-0.5*np.pi] * cfg_random["features_number"],
    [0.5*np.pi] * cfg_random["features_number"]   
)
pqc_cfg.update({
    "base_frecuency" : base_frecuency,
    "shift_feature" : shift_feature    
})   
print(pqc_cfg)
pqc, weights_names, features_names = hardware_efficient_ansatz(**pqc_cfg)
observable = z_observable(**pqc_cfg)

In [ ]:
# plot PQC
circuit = pqc.to_circ()
circuit.display()

## 3. QPU info

To evaluate the different quantum circuits,  a **myQLM QPU** is needed. In the following cell, the configuration for it is loaded.

In [ ]:
from QQuantLib.utils.benchmark_utils import combination_for_list
from QQuantLib.qpu.select_qpu import select_qpu

In [ ]:
json_qpu = "../../benchmark/qml4var/JSONs/qpu_ideal.json"
with open(json_qpu) as json_file:
    qpu_dict = json.load(json_file)
qpu_list = combination_for_list(qpu_dict)
qpu_dict = qpu_list[0]
print(qpu_dict)

## 4. Get the workflow evaluation functions

To compute the *Loss function* the different workflows for evaluating the **CDF** and the **PDF** using **PQC**s will be needed:

In [ ]:
from QQuantLib.qml4var.myqlm_workflows import cdf_workflow, pdf_workflow, workflow_execution

In [ ]:
# Configuration for workflows
nbshots = 0
workflow_cfg = {
    "pqc" : pqc,
    "observable" : observable,
    "weights_names" : weights_names,
    "features_names" : features_names,
    "nbshots" : nbshots,
    "qpu_info" : qpu_dict
}

In [ ]:
# First configure porperly the desired workflow computation using Python lambdas

# For computing CDF using PQC
cdf_workflow_ = lambda w,x : cdf_workflow(w, x, **workflow_cfg)
# For computing PDF using PQC
pdf_workflow_ = lambda w,x : pdf_workflow(w, x, **workflow_cfg)

## 5. The Loss function


The main objective is to train a **PQC** represented by $F^*(\textbf{x}, \theta)$ using data sampled from a **CDF** financial distribution: $\tilde{\textbf{x}}^j \sim F(\textbf{x})$ in such a way that the trained $F^*(\textbf{x}, \theta)$ can be used as a surrogate model of the $F(\textbf{x})$ for **VaR** computations.

The *Loss function* we are going to use to achieve this objective will be the following one:

$$R_{L^2, \bar{L}^2}^{S_{\chi}} = \alpha_0 \frac{1}{m}\sum_{j=0}^{m-1} (cdf(\tilde{\textbf{x}}^j; \theta) -y^j)^2 + \alpha_1 \left(-\frac{1}{m}\sum_{j=0}^{m-1} pdf(\tilde{\textbf{x}}^j; \vec{\theta})  + \int_{\textbf{x}_{min}}^{\textbf{x}_{max}} pdf^2(\textbf{x}; \theta) d\textbf{x} \right) $$

Where:
 
* $cdf(\tilde{\textbf{x}}^j; \theta)$ is the **CDF** computed by the *PQC* evaluated in the training data $\tilde{\textbf{x}}^j$.
* $pdf(\tilde{\textbf{x}}^j; \theta)$ is the **PDF** computed using the *PQC* evaluated in the training data $\tilde{\textbf{x}}^j$.

The integral can be evaluated numerically by discretizing the complete domain and computing for each point $\textbf{x}^k$ the $pdf(\textbf{x}^k; \theta)$

This loss is defined into the function *loss_function_qdml* from the **QQuantLib.qml4var.losses** module. The mandatory inputs are:

* labels: they are the $y^j$
* predict_cdf : they are the $cdf(\textbf{x}^j; \theta)$
* predict_pdf : they are the $pdf(\textbf{x}^j; \theta)$
* integral: the evaluation of the integral:$\int_{\textbf{x}_{min}}^{\textbf{x}_{max}} pdf^2(\textbf{x}; \theta) d\textbf{x} $
* loss_weights: list wit the weights for each part of the loss function: [$\alpha_0$, $\alpha_1$]

All this inputs can be computed using the *workflow_execution* (from **QQuantLib.qml4var.qlm_procces**) using a properly configured *cdf_workflow* and *pdf_workflow* functions

In [ ]:
from QQuantLib.qml4var.losses import loss_function_qdml

### 5.1 Computation of the inputs for loss_function_qdml


In the following cells, the computations for $cdf(\tilde{\textbf{x}}^j; \theta)$  and for $pdf(\tilde{\textbf{x}}^j; \theta)$ are done:

In [ ]:
%%time
# initialize weights
weights_= [np.random.rand() for w in weights_names]
#get cdf evaluated over the training dataset
cdf_prediction = np.array(workflow_execution(weights_, x_train, cdf_workflow_))
#get pdf evaluated over the training dataset
pdf_prediction = np.array(workflow_execution(weights_, x_train, pdf_workflow_))
# Rearrange using the shape of the labels
cdf_prediction = cdf_prediction.reshape(y_train.shape)
pdf_prediction = pdf_prediction.reshape(y_train.shape)

Now we need to build the domain for computing the mandatory integral for creating the desired *Loss function*. 

The domain will be discretized over a given number of points for each axis (i.e. for each possible feature). For more than 1 feature, the domain will be the cartesian product of the discretization over all the features (we are going to build a complete mesh over all the n-dimensional domain)

In [ ]:
# Discretization over each feature domain.
discretization_points = 100
domain_x = np.linspace(
    [cfg_random["minval"]] * cfg_random["features_number"],
    [cfg_random["maxval"]] * cfg_random["features_number"],
    discretization_points
)
# Cartesian product for building a mesh over the n-dimensional domain
domain_x = np.array(list(
    product(*[domain_x[:, i] for i in range(domain_x.shape[1])])
))
domain_x.shape

Now we are going to obtain the **PDF** evaluation for each n-dimensional point of the domain! For this a new *workflow* will be generated:

In [ ]:
pdf_workflow_square = lambda w, x: pdf_workflow(w, x, **workflow_cfg) ** 2

In [ ]:
# Prediction of PDF using x_quad: 
pdf_squqare_domain_prediction = np.array(workflow_execution(weights_, domain_x, pdf_workflow_square))

#### Computing the integral

Now we have all ingredients for computing the integral mandatory for computing the *loss function*. The function *compute_integral* from **QQuantLib.qml4var.losses** module can be used for computing it. 

The inputs of the function are:

* y_array: numpy array or list of dask futures with the y for integral computation
    * For numpy array expected shape is: (n, 1)
* x_array: numpy array with the domain for the numerical integration:
    * Expected shape: (n, features)
* dask_client: Dask client to speed up computations.

The output will be a float if dask_client is not provided and a future otherwise.

**Computation considerations**

The integral computation will depend on the inputs:
* If x_array.shape == (n, 1) then np.trapz is used for integral computation.
* If x_array.shape == (n, 2) AND dask_client is not provided then the double integration is performed using np.trapz by meshgrid properly the domain and doing the corresponding reshape of the y_array
* If x_array.shape == (n, 2) AND dask_client is provided then the integration is computed using the MonteCarlo integration method.
* If x_array.shape == (n, >2) then the integration is computed using the MonteCarlo integration method.



In [ ]:
from QQuantLib.qml4var.losses import compute_integral

In [ ]:
def compute_integral(y_array, x_array, dask_client=None):
    """
    Function for computing numerical integral of inputs arrays. Considerations:
    * if x_array has shape(n, 1) then numpy trapz is used for computing integral.
    * if x_array has shape(n, 2) and dask_client is None numpy trapz
        is used for computing the double integral.
    * if x_array has shape(n, 2) and dask_client is provided then MonteCarlo
        integration is used
    * if x_array has shape(n, > 2) MonteCarlo integration is used
    Parameters
    ----------
    y_array : np.array or dask futures:
        array or futures (only is dask_client is provided) with the
        y-values for integration.  For array expected shape is: shape(n)
    x_array : np array
        array with the x domain for integration: Shape(n, n_features)
    dask_client : DASK client
        DASK client to submit computation of the integral.
        y_array MUST BE a list of futures
    Returns
    -------
    integral : float or future
        float or future (if dask_client is provided) with the desired
        integral computation.
    """
    if x_array.shape[1] == 1:
        if dask_client is None:
            integral = np.trapz(y=y_array, x=x_array[:, 0])
        else:
            integral = dask_client.submit(np.trapz, y_array, x_array[:, 0])
    elif x_array.shape[1] == 2:
        if dask_client is None:
            x_domain, y_domain = np.meshgrid(
                np.unique(x_array[:, 0]),
                np.unique(x_array[:, 1])
            )
            y_array_ = y_array.reshape(x_domain.shape)
            integral = np.trapz(
                np.trapz(y=y_array_, x=x_domain),
                x=y_domain[:, 0]
            )
        else:
            # MonteCarlo integration
            factor = np.prod(x_array.max(axis=0) - x_array.min(axis=0)) / len(y_array)
            integral = dask_client.submit(
                lambda x: np.sum(x) * factor,
                y_array
            )

    else:
        # MonteCarlo integration
        if dask_client is None:
            factor = np.prod(x_array.max(axis=0) - x_array.min(axis=0)) / y_array.size
            integral = np.sum(y_array) * factor
        else:
            factor = np.prod(x_array.max(axis=0) - x_array.min(axis=0)) / len(y_array)
            integral = dask_client.submit(
                lambda x: np.sum(x) * factor,
                y_array
            )
    return integral

In [ ]:
integral = compute_integral(pdf_squqare_domain_prediction, domain_x)
print("integral : {}".format(integral))

In [ ]:
loss_ = loss_function_qdml(
    y_train, cdf_prediction, pdf_prediction, integral
)
print("The computed Loss is: {}".format(loss_))

#### Using Dask Client

The number of quantum circuits evaluations can scale very quicly (especially if more tha 2 features are used). If it is available a *DASK* cluster can be used to speed up this computations. As explained in notebook *17_qml4var_pqc_evaluation.ipynb* the *DASK* client should be provided to the *workflow_execution* function!!


**BE AWARE**

The following cells should be executed only if a *DASK* cluster is available.

In [ ]:
from distributed import Client
path_to_schedule_json = "/home/cesga/gferro/Codigo/qlm_cVar/dask_cluster_ft3/scheduler_info.json"
#path_to_schedule_json = "/home/cesga/gferro/Codigo/dask_cluster_ft3/scheduler_info.json"
client = Client(
    scheduler_file = path_to_schedule_json,
)

In [ ]:
%%time
#get cdf evaluated over the training dataset
cdf_prediction_dask = workflow_execution(weights_, x_train, cdf_workflow_, client)
#get pdf evaluated over the training dataset
pdf_prediction_dask = workflow_execution(weights_, x_train, pdf_workflow_, client)
# Discretization over each feature domain.
discretization_points = 100
domain_x = np.linspace(
    [cfg_random["minval"]] * cfg_random["features_number"],
    [cfg_random["maxval"]] * cfg_random["features_number"],
    discretization_points
)
# Cartesian product for building a mesh over the n-dimensional domain
domain_x = np.array(list(
    product(*[domain_x[:, i] for i in range(domain_x.shape[1])])
))

# Prediction for integration
pdf_squqare_domain_prediction_dask = workflow_execution(weights_, domain_x, pdf_workflow_square, client)
# The compute_integral need the futures in this case
dask_integral = compute_integral(pdf_squqare_domain_prediction_dask, domain_x, client)

In [ ]:
#IT IS A FUTURE
dask_integral

Once all the futures are obtained we need to gather all the features!!

In [ ]:
# cdf_prediction_dask, pdf_prediction_dask and pdf_domain_prediction_dask are dask futures. We need to
# retrieve the results of the computations from dask cluster using a gather

cdf_prediction_dask = np.array(client.gather(cdf_prediction_dask))
pdf_prediction_dask = np.array(client.gather(pdf_prediction_dask))
cdf_prediction_dask = cdf_prediction_dask.reshape(y_train.shape)
pdf_prediction_dask = pdf_prediction_dask.reshape(y_train.shape)

# Additionally we retrieve the integral
dask_integral = client.gather(dask_integral)

In [ ]:
print(np.isclose(cdf_prediction, cdf_prediction_dask).all())
print(np.isclose(pdf_prediction, pdf_prediction_dask).all())

In [ ]:
print("integral: {}. dask_integral: {}".format(integral, dask_integral))

In [ ]:
#Now the Loss function can be computed
loss_from_dask = loss_function_qdml(
    y_train, cdf_prediction_dask, pdf_prediction_dask, dask_integral
)
print("The computed Loss is: {}".format(loss_from_dask))

### 5.2 The workflow_for_qdml function

The *workflow_for_qdml* function from **QQuantLib.qml4var.myqlm_workflows** module, implements the before explained scheme to obtain easily the mandatory inputs for the *loss_function_qdml*.

The inputs are:
* weights: The weights for the **PQC** ($\theta$).
* data_x: numpy array with the training features ($\{\tilde{\textbf{x}}^j ; j=0, 1,\cdots m-1\}$).
    * Shape: (-1, number of features)
* data_y:numpy array with the training features or targets.
    * Shape: (-1, number of 1). 
* kwargs : Keyword arguments with additional information. Mandatory  keywords:
    * pqc: The value MUST BE the **myQLM Program** that implements the desired PQC.
    * observable: The value MUST BE the **myQLM Observable** for the PQC.
    * weights_names: The value MUST BE a list of all the parameter names of the **PQC** related to the weights.
    * features_names: The value MUST BE a list of all the parameter names of the **PQC** related to the features.
    * nbshots: The number of shots for evaluating the **PQC**.
    * qpu_info: configuration dictionary for QPU
    * minval: list with minimum values of the domain for all the features.
    * maxval: list with maximum values of the domain for all the features.
    * points: discretization number of points for the domain of 1 feature.

The return of a function will be a Python dictionary with the following keys:

* data_y : input data_y data
* y_predict_cdf : CDF prediction for data_x
* y_predict_pdf : PDF prediction for data_x
* integral : the computed integral

In [ ]:
from QQuantLib.qml4var.myqlm_workflows import workflow_for_qdml

In [ ]:
discretization_points = 100
workflow_cfg = {
    "pqc" : pqc,
    "observable" : observable,
    "weights_names" : weights_names,
    "features_names" : features_names,
    "nbshots" : nbshots,
    "qpu_info" : qpu_dict,
    "minval" : [cfg_random["minval"]] * cfg_random["features_number"],
    "maxval" : [cfg_random["maxval"]] * cfg_random["features_number"],
    "points" : discretization_points,
}

In [ ]:
%%time
data = workflow_for_qdml(weights_, x_train, y_train, **workflow_cfg)
cdf_tp = data["y_predict_cdf"]
pdf_tp = data["y_predict_pdf"]

In [ ]:
np.isclose(cdf_tp, cdf_prediction).all()

In [ ]:
np.isclose(pdf_tp, pdf_prediction).all()

In [ ]:
np.isclose(data["integral"], integral)

#### DASK client

To the **workflow_for_qdml** a **DASK** client can be passed to speed up computation.

**BE AWARE**

The following cells should be executed only if a Dask cluster is available

In [ ]:
%%time
data_dask = workflow_for_qdml(weights_, x_train, y_train, dask_client=client, **workflow_cfg)
cdf_tp_dask = data_dask["y_predict_cdf"]
pdf_tp_dask = data_dask["y_predict_pdf"]
integral_dask = data_dask["integral"]

In [ ]:
np.isclose(cdf_tp, cdf_tp_dask).all()

In [ ]:
np.isclose(pdf_tp, pdf_tp_dask).all()

In [ ]:
np.isclose(data["integral"], integral_dask)

### 5.3. qdml_loss_workflow function

To compute directly the desired *Loss function* the *qdml_loss_workflow* function from QQuantLib.qml4var.myqlm_workflows. This function uses the *workflow_for_qdml* function and pass the outputs to the *loss_function_qdml* from **QQuantLib.qml4var.losses** module. Do the loss computation can be done transparently.


In [ ]:
from QQuantLib.qml4var.myqlm_workflows import qdml_loss_workflow

In [ ]:
%%time
loss_ = qdml_loss_workflow(weights_, x_train, y_train, **workflow_cfg)

In [ ]:
print("Loss function: {}".format(loss_))

#### DASK client

To the **qdml_loss_workflow** a **DASK** client can be passed to speed up computation.

**BE AWARE**

The following cells should be executed only if a *DASK* cluster is available.

In [ ]:
%%time
loss_dask = qdml_loss_workflow(weights_, x_train, y_train, dask_client=client, **workflow_cfg)

In [ ]:
print("Loss function using dask: {}".format(loss_dask))

In [ ]:
loss_dask

### 5.4 The mse_workflow function

Using the explained procedures along this notebook the user can define their own losses and use the different functions for building a *workflow* for evaluating them. The *mse_workflow* from **QQuantLib.qml4var.myqlm_workflows** builds this *workflow* for computing directly a *Mean Square Error* loss function.

(The *DASK* client can be provided to the *mse_workflow* function)

In [ ]:
from QQuantLib.qml4var.myqlm_workflows import mse_workflow

In [ ]:
mse_ = mse_workflow(weights_, x_train, y_train, dask_client=None, **workflow_cfg)
print(mse_)

## 6. Numeric Gradients

For training the **PQC** in adddition to a *Loss function* a function for computing gradients it is mandatory. 

In our code, the user can use the *numeric_gradient* from **QQuantLib.qml4var.losses** module. This function allows to compute the gradients of a properly configured loss.

The inputs are:

* weights : this is the $\vec{\theta}$ for the **PQC**
* data_x : the dataset with the features: $\vec{x}^j$
* data_y : the labels of the dataset: $y^j$
* loss: this is a loss function (like the *mse_workflow* or the *qdml_loss_workflow* from **training_functions**) properly configured. This function only should recived *weights*, *data_x* and *data_y*.

In [ ]:
from QQuantLib.qml4var.losses import numeric_gradient

First we need to porperly configured the loss function in such a way that only can receives the *weights*, *data_x* and *data_y*. To do that we can use the **lambda** Python functionality as shown in the following cell:

In [ ]:
# We are going  to set all the arguments except the *weights*, *data_x* and *data_y*
loss_function_ = lambda w_, x_, y_ : qdml_loss_workflow(
    w_, x_, y_, dask_client=None, **workflow_cfg)

loss_function_mse = lambda w_, x_, y_ : mse_workflow(
    w_, x_, y_, dask_client=None, **workflow_cfg)

Now we can provide the before lambda functions to the *numeric_gradient* for computing the corresponding gradients.

In [ ]:
%%time
gradients_loss = numeric_gradient(weights_, x_train, y_train, loss_function_)

In [ ]:
%%time
gradients_mse = numeric_gradient(weights_, x_train, y_train, loss_function_mse)

#### DASK client

The computation of gradients involves a lot of executions. In this case, a *DASK* cluster can speed up the computations dramatically. You need to configure properly the loss functions workflows for using a *DASK* cluster to speed up the computations.

**BE AWARE**

The following cells should be executed only if a *DASK* cluster is available.

In [ ]:
# We are going  to set all the arguments except the *weights*, *data_x* and *data_y*
loss_function_dask = lambda w_, x_, y_ : qdml_loss_workflow(
    w_, x_, y_, dask_client=client, **workflow_cfg)

loss_function_mse_dask = lambda w_, x_, y_ : mse_workflow(
    w_, x_, y_, dask_client=client, **workflow_cfg)

In [ ]:
%%time
gradients_loss_dask = numeric_gradient(weights_, x_train, y_train, loss_function_dask)

In [ ]:
np.isclose(gradients_loss_dask, gradients_loss).all()

In [ ]:
%%time
gradients_mse_dask = numeric_gradient(weights_, x_train, y_train, loss_function_mse_dask)

In [ ]:
np.isclose(gradients_mse, gradients_mse_dask).all()

## 8. Evaluation of PQCs with 2 features


Mandatory evaluation of PQCs when dataset has more than 1 feature can be very intensive and a Dask Cluster is recommended for such computations

In [ ]:
from benchmark.qml4var.data_sets import create_random_data

### get data

In [ ]:
# Get Data with several features
cfg_2d_random = {
    "n_points_train": 50, 
    "n_points_test" : 100,
    "minval" : -np.pi,
    "maxval" : np.pi,
    "features_number" : 2
}
x_train_2d, y_train_2d, x_test_2d, y_test_2d = create_random_data(
    **cfg_2d_random
)

In [ ]:
%matplotlib inline
# Only for 2D
fig = plt.figure()
ax1 = fig.add_subplot(projection='3d')

ax1.plot3D(x_train_2d[:, 0], x_train_2d[:, 1], y_train_2d[:, 0], 'o')
ax1.plot3D(x_test_2d[:, 0], x_test_2d[:, 1], y_test_2d[:, 0], '-', alpha=0.6)
ax1.view_init(elev=21, azim=180)

### get QPU info

In [ ]:
from QQuantLib.utils.benchmark_utils import combination_for_list
from QQuantLib.qpu.select_qpu import select_qpu

In [ ]:
json_qpu = "../../benchmark/qml4var/JSONs/qpu_ideal.json"
with open(json_qpu) as json_file:
    qpu_dict = json.load(json_file)
qpu_list = combination_for_list(qpu_dict)
qpu_dict = qpu_list[0]
print(qpu_dict)

### get PQC

In [ ]:
from QQuantLib.qml4var.architectures import hardware_efficient_ansatz, z_observable, normalize_data

In [ ]:
# PQC building
pqc_cfg_2d = {
    "features_number" : cfg_2d_random["features_number"],
    "n_qubits_by_feature" : 2,
    "n_layers": 3    
}
# Normalization function
base_frecuency, shift_feature = normalize_data(
    [cfg_2d_random["minval"]] * cfg_2d_random["features_number"],
    [cfg_2d_random["maxval"]] * cfg_2d_random["features_number"],
    [-0.5*np.pi] * cfg_2d_random["features_number"],
    [0.5*np.pi] * cfg_2d_random["features_number"]   
)
pqc_cfg_2d.update({
    "base_frecuency" : base_frecuency,
    "shift_feature" : shift_feature    
})   
print(pqc_cfg_2d)
pqc_2d, weights_names_2d, features_names_2d = hardware_efficient_ansatz(**pqc_cfg_2d)
observable_2d = z_observable(**pqc_cfg_2d)
# initialize weights
weights_2d= [np.random.rand() for w in weights_names_2d]
discretization_points = 100
nbshots = 0
workflow_cfg_2d = {
    "pqc" : pqc_2d,
    "observable" : observable_2d,
    "weights_names" : weights_names_2d,
    "features_names" : features_names_2d,
    "nbshots" : nbshots,
    "qpu_info" : qpu_dict,
    "minval" : [cfg_2d_random["minval"]] * cfg_2d_random["features_number"],
    "maxval" : [cfg_2d_random["maxval"]] * cfg_2d_random["features_number"],
    "points" : discretization_points,
}

In [ ]:
# plot PQC
circuit = pqc_2d.to_circ()
circuit.display()

### Workflows

In [ ]:
from QQuantLib.qml4var.myqlm_workflows import qdml_loss_workflow

### dask client

In [ ]:
from distributed import Client
path_to_schedule_json = "/home/cesga/gferro/Codigo/qlm_cVar/dask_cluster_ft3/scheduler_info.json"
#path_to_schedule_json = "/home/cesga/gferro/Codigo/dask_cluster_ft3/scheduler_info.json"
client = Client(
    scheduler_file = path_to_schedule_json,
)

In [ ]:
# Configure loss_function_dask
loss_function_dask = lambda w_, x_, y_ : qdml_loss_workflow(
    w_, x_, y_, dask_client=client, **workflow_cfg_2d)

In [ ]:
%%time
loss_2d_dask = loss_function_dask(weights_2d, x_train_2d, y_train_2d)

In [ ]:
loss_2d_dask

In [ ]:
loss_function_no_dask = lambda w_, x_, y_ : qdml_loss_workflow(
    w_, x_, y_, **workflow_cfg_2d)

In [ ]:
%%time
loss_2d_no_dask = loss_function_no_dask(weights_2d, x_train_2d, y_train_2d)
print(loss_2d_no_dask)